In [0]:
import keras
import pickle
import gzip
import pandas as pd
import datetime
import holidays
import gensim
import numpy as np
from gensim.models.doc2vec import TaggedDocument
from keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


In [4]:
reddit_model = load_model("./drive/My Drive/3classmodel.model")
sentiment_model = pickle.load(gzip.open("./drive/My Drive/compressed_sentiment_classifier", "rb"))
vector_model = gensim.models.Doc2Vec.load("/content/drive/My Drive/doc2vec.model")

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SelectKBest from version 0.22 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarnin

In [0]:
us_holidays = {}
for date, name in sorted(holidays.US(years=[2017, 2018, 2019, 2020], observed = False).items()):
    if name not in us_holidays:
        us_holidays[name] = [date]
    else:
        us_holidays[name].append(date)
days_of_week = [0, 1, 2, 3, 4, 5, 6]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
time_stamps = [1, 5, 9, 13, 17, 21]


In [0]:
text = "Would you take a 50/50 chance at $5,000,000 or ? Why or why not? fuck "
date = pd.Timestamp('2019-01-01 00:25:46.376400')

In [11]:
post_vector = vector_model.infer_vector(text)
# sentiment = np.array(sentiment_model.predict_proba([text])[0][1])
sentiment = np.array(sentiment_model.predict_proba([text]))
contains_reddit = [int('reddit' in text)]

holidays_vector = []
for day in us_holidays:
  holidays_vector.append(int(date in us_holidays[day]))

days_vector = []
for day in days_of_week:
  days_vector.append(int(date.dayofweek == day))

months_vector = []
for month in months:
  months_vector.append(int(date.month == month))

time_stamp_vector = []
for i in range(len(time_stamps)):
  if i == 5:
    time_stamp_vector.append(int(21 <= date.hour <= 24 or date.hour < 1))
    break
  time_stamp_vector.append(int(time_stamps[i] <= date.hour < time_stamps[i+1]))

len_vector = [(len(text)/300)]

lst = contains_reddit + holidays_vector + days_vector + months_vector + time_stamp_vector + len_vector
a = np.concatenate((post_vector, sentiment), axis = None)
b = np.array(lst)
input_vector = np.concatenate((a, b), axis = None)
print(sentiment)

[[0.22601 0.77399]]


In [1]:
reddit_model.predict(np.array([input_vector]))


NameError: ignored